# INTRO 
[VERY MUCH STILL IN PROGRESS]
- Intro text
- Experiment with shorter videos (make each section a video)
- Form 4 Parser


- Track meta info like name and issuer.
- And more changeable stuff like relationship, ownership, attorney
- Additional stuff
    - Scrape contracts to find legal boilerplate and build index
        - https://www.sec.gov/Archives/edgar/data/0001527709/000119312511351200/d272812dex101.htm
    - other business intel?

In [1]:
import requests
from secret_services import *

In [2]:
#Edgar get was loaded in secret_servcies
cik_data=edgar_get("https://www.sec.gov/Archives/edgar/cik-lookup-data.txt")
cik_data

<Response [200]>

In [3]:
cik_lookup={}
for line in cik_data.text.splitlines():
    split=line.split(":")
    cik_lookup[split[0]]=split[1]
cik_lookup

{'!J INC': '0001438823',
 '#1 A LIFESAFER HOLDINGS, INC.': '0001509607',
 '#1 ARIZONA DISCOUNT PROPERTIES LLC': '0001457512',
 '#1 PAINTBALL CORP': '0001433777',
 '$ LLC': '0001427189',
 '$AVY, INC.': '0001655250',
 '& S MEDIA GROUP LLC': '0001447162',
 '&TV COMMUNICATIONS INC.': '0001479357',
 '&VEST DOMESTIC FUND II KPIV, L.P.': '0001802417',
 '&VEST DOMESTIC FUND II LP': '0001800903',
 '&VEST OFFSHORE FUND II L.P.': '0001800902',
 "'LAINE'S BAKE SHOP LLC, SERIES OF BG CONSORTIUM LLC": '0001732207',
 "'MERICAN MULE, INC.": '0001833137',
 "'MKTG, INC.'": '0000886475',
 "'OHANA LABS INC.": '0001703629',
 "'PARAGON VMS USA CORP": '0001786139',
 '(OURCROWD INVESTMENT IN MST) L.P.': '0001599496',
 '(Y.Z) QUEENCO LTD.': '0001623088',
 '.406 VENTURES I GP, L.P.': '0001854758',
 '.406 VENTURES I GP, LLC': '0001854757',
 '.406 VENTURES MANAGEMENT, L.P.': '0001766976',
 '.CLUB DOMAINS, LLC': '0001577787',
 '001 - SERIES OF IPOSHARKS VENTURE MASTER FUND, LLC': '0001863872',
 '002 - SERIES OF IP

In [7]:
len(cik_lookup)

827599

Scraping every single name would take a long time. Rather than try that we scrape the insider trading reports from a few prominent companies and then do a form of page rank to find entities of interest that are engaged in insider trading!

In [8]:
resp=edgar_get("https://www.sec.gov/files/company_tickers.json")
resp.text

'{"0":{"cik_str":320193,"ticker":"AAPL","title":"Apple Inc."},"1":{"cik_str":789019,"ticker":"MSFT","title":"MICROSOFT CORP"},"2":{"cik_str":1652044,"ticker":"GOOG","title":"Alphabet Inc."},"3":{"cik_str":1018724,"ticker":"AMZN","title":"AMAZON COM INC"},"4":{"cik_str":1318605,"ticker":"TSLA","title":"Tesla, Inc."},"5":{"cik_str":1326801,"ticker":"FB","title":"Meta Platforms, Inc."},"6":{"cik_str":40545,"ticker":"GE","title":"GENERAL ELECTRIC CO"},"7":{"cik_str":1067983,"ticker":"BRK-A","title":"BERKSHIRE HATHAWAY INC"},"8":{"cik_str":1046179,"ticker":"TSM","title":"TAIWAN SEMICONDUCTOR MANUFACTURING CO LTD"},"9":{"cik_str":19617,"ticker":"JPM","title":"JPMORGAN CHASE & CO"},"10":{"cik_str":1403161,"ticker":"V","title":"VISA INC."},"11":{"cik_str":354950,"ticker":"HD","title":"HOME DEPOT, INC."},"12":{"cik_str":884394,"ticker":"SPY","title":"SPDR S&P 500 ETF TRUST"},"13":{"cik_str":200406,"ticker":"JNJ","title":"JOHNSON & JOHNSON"},"14":{"cik_str":731766,"ticker":"UNH","title":"UNITEDH

In [9]:
import pandas as pd
ticker_lookup=pd.DataFrame(resp.json()).T.set_index("ticker")
ticker_lookup

,cik_str,title
ticker,,
AAPL,320193,Apple Inc.
MSFT,789019,MICROSOFT CORP
GOOG,1652044,Alphabet Inc.
AMZN,1018724,AMAZON COM INC
TSLA,1318605,"Tesla, Inc."
...,...,...
PCXCU,1855751,Parsec Capital Acquisitions Corp.
LILMW,1855756,Lilium N.V.
ENERU,1855555,ACCRETION ACQUISITION CORP.


In [10]:
#looks to be sorted by by some size related metric already!
ticker_lookup.iloc[:5]

,cik_str,title
ticker,,
AAPL,320193,Apple Inc.
MSFT,789019,MICROSOFT CORP
GOOG,1652044,Alphabet Inc.
AMZN,1018724,AMAZON COM INC
TSLA,1318605,"Tesla, Inc."


In [11]:
#fix padding on str
ticker_lookup['cik_str']=ticker_lookup['cik_str'].apply(lambda x: str(x).zfill(10))
ticker_lookup

,cik_str,title
ticker,,
AAPL,0000320193,Apple Inc.
MSFT,0000789019,MICROSOFT CORP
GOOG,0001652044,Alphabet Inc.
AMZN,0001018724,AMAZON COM INC
TSLA,0001318605,"Tesla, Inc."
...,...,...
PCXCU,0001855751,Parsec Capital Acquisitions Corp.
LILMW,0001855756,Lilium N.V.
ENERU,0001855555,ACCRETION ACQUISITION CORP.


In [14]:
ciks_of_interest=ticker_lookup['cik_str'].iloc[:5]
ciks_of_interest

ticker
AAPL    0000320193
MSFT    0000789019
GOOG    0001652044
AMZN    0001018724
TSLA    0001318605
Name: cik_str, dtype: object

In [15]:
[(x,cik_lookup[x]) for x in cik_lookup if "APPLE INC" in x]

[('APPLE INC.', '0000320193'),
 ('APPLE INC', '0000320193'),
 ('GOLDEN APPLE INCOME INC', '0001269529'),
 ('LINEAPPLE INC', '0001614084'),
 ('SCAPPLE INC', '0001252277')]

In [16]:
cik=ciks_of_interest[0]

In [17]:
url=f"https://data.sec.gov/submissions/CIK{cik}.json"
print(url)

https://data.sec.gov/submissions/CIK0000320193.json


In [19]:
resp=edgar_get(url)

In [20]:
company_index=resp.json()
filings=pd.DataFrame(company_index['filings']['recent'])
filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000320193-21-000114,2021-11-16,2021-11-12,2021-11-16T18:30:25.000Z,,4,,,,4609,0,0,xslF345X03/wf-form4_163710541081314.xml,FORM 4
1,0000320193-21-000112,2021-11-12,2021-11-09,2021-11-12T18:32:23.000Z,,4,,,,5680,0,0,xslF345X03/wf-form4_163675992058856.xml,FORM 4
2,0000320193-21-000111,2021-11-12,2021-11-09,2021-11-12T18:30:48.000Z,,3,,,,7973,0,0,xslF345X02/wf-form3_163675983081322.xml,FORM 3
3,0001193125-21-328151,2021-11-12,2021-11-09,2021-11-12T16:31:14.000Z,34,8-K,001-36743,211404083,"5.02,5.03,9.01",492640,1,1,d259993d8k.htm,8-K
4,0000320193-21-000108,2021-11-03,2021-11-01,2021-11-03T18:31:28.000Z,,4,,,,7070,0,0,xslF345X03/wf-form4_163597867263388.xml,FORM 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998,0001181431-11-053172,2011-10-20,2011-10-18,2011-10-20T19:00:39.000Z,,4,,,,25001,0,0,xslF345X03/rrd322715.xml,FORM 4: J. WILLIAMS 10/18/2011 RSU VEST AND 1...
999,0001181431-11-053007,2011-10-18,2011-10-15,2011-10-18T19:01:49.000Z,,4,,,,23017,0,0,xslF345X03/rrd322657.xml,FORM 4: J. WILLIAMS 10/15/2011 RSU VEST AND 1...
1000,0001181431-11-053006,2011-10-18,2011-10-15,2011-10-18T19:01:19.000Z,,4,,,,15693,0,0,xslF345X03/rrd322655.xml,FORM 4: B. RAFAEL 10/15/2011 RSU VEST
1001,0001181431-11-053005,2011-10-18,2011-10-15,2011-10-18T19:00:52.000Z,,4,,,,14335,0,0,xslF345X03/rrd322659.xml,FORM 4: E. CUE 10/15/2011 RSU VEST


# Filter These to Insider Transactions
https://www.sec.gov/files/forms-3-4-5.pdf

In [21]:
insider_filings=filings[filings['form'].isin(["3","3/A","4","4/A","5","5/A"])]
insider_filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000320193-21-000114,2021-11-16,2021-11-12,2021-11-16T18:30:25.000Z,,4,,,,4609,0,0,xslF345X03/wf-form4_163710541081314.xml,FORM 4
1,0000320193-21-000112,2021-11-12,2021-11-09,2021-11-12T18:32:23.000Z,,4,,,,5680,0,0,xslF345X03/wf-form4_163675992058856.xml,FORM 4
2,0000320193-21-000111,2021-11-12,2021-11-09,2021-11-12T18:30:48.000Z,,3,,,,7973,0,0,xslF345X02/wf-form3_163675983081322.xml,FORM 3
4,0000320193-21-000108,2021-11-03,2021-11-01,2021-11-03T18:31:28.000Z,,4,,,,7070,0,0,xslF345X03/wf-form4_163597867263388.xml,FORM 4
9,0000320193-21-000099,2021-10-21,2021-10-19,2021-10-21T18:30:48.000Z,,4,,,,5213,0,0,xslF345X03/wf-form4_163485542776887.xml,FORM 4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,0001181431-11-053173,2011-10-20,2011-10-15,2011-10-20T19:03:24.000Z,,4,,,,13011,0,0,xslF345X03/rrd322662.xml,FORM 4: E. CUE 10/18/2011 RSU VEST
998,0001181431-11-053172,2011-10-20,2011-10-18,2011-10-20T19:00:39.000Z,,4,,,,25001,0,0,xslF345X03/rrd322715.xml,FORM 4: J. WILLIAMS 10/18/2011 RSU VEST AND 1...
999,0001181431-11-053007,2011-10-18,2011-10-15,2011-10-18T19:01:49.000Z,,4,,,,23017,0,0,xslF345X03/rrd322657.xml,FORM 4: J. WILLIAMS 10/15/2011 RSU VEST AND 1...
1000,0001181431-11-053006,2011-10-18,2011-10-15,2011-10-18T19:01:19.000Z,,4,,,,15693,0,0,xslF345X03/rrd322655.xml,FORM 4: B. RAFAEL 10/15/2011 RSU VEST


In [24]:
filing=filings.iloc[0]
filing

accessionNumber                             0000320193-21-000114
filingDate                                            2021-11-16
reportDate                                            2021-11-12
acceptanceDateTime                      2021-11-16T18:30:25.000Z
act                                                             
form                                                           4
fileNumber                                                      
filmNumber                                                      
items                                                           
size                                                        4609
isXBRL                                                         0
isInlineXBRL                                                   0
primaryDocument          xslF345X03/wf-form4_163710541081314.xml
primaryDocDescription                                     FORM 4
Name: 0, dtype: object

In [25]:
doc_url=f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{filing['accessionNumber'].replace('-','')}/{filing['primaryDocument'].split('/')[-1]}"
resp=edgar_get(doc_url)
resp

<Response [200]>

In [26]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(resp.text,"xml")
soup

<?xml version="1.0" encoding="utf-8"?>
<ownershipDocument>
<schemaVersion>X0306</schemaVersion>
<documentType>4</documentType>
<periodOfReport>2021-11-12</periodOfReport>
<notSubjectToSection16>0</notSubjectToSection16>
<issuer>
<issuerCik>0000320193</issuerCik>
<issuerName>Apple Inc.</issuerName>
<issuerTradingSymbol>AAPL</issuerTradingSymbol>
</issuer>
<reportingOwner>
<reportingOwnerId>
<rptOwnerCik>0001631982</rptOwnerCik>
<rptOwnerName>KONDO CHRIS</rptOwnerName>
</reportingOwnerId>
<reportingOwnerAddress>
<rptOwnerStreet1>ONE APPLE PARK WAY</rptOwnerStreet1>
<rptOwnerStreet2/>
<rptOwnerCity>CUPERTINO</rptOwnerCity>
<rptOwnerState>CA</rptOwnerState>
<rptOwnerZipCode>95014</rptOwnerZipCode>
<rptOwnerStateDescription/>
</reportingOwnerAddress>
<reportingOwnerRelationship>
<isDirector>0</isDirector>
<isOfficer>1</isOfficer>
<isTenPercentOwner>0</isTenPercentOwner>
<isOther>0</isOther>
<officerTitle>Principal Accounting Officer</officerTitle>
<otherText/>
</reportingOwnerRelationship>


- First we'll parse all forms
- then we'll decide which insiders to prioritize downloading all their submissions

In [27]:
cik="0001631982"
url=f"https://data.sec.gov/submissions/CIK{cik}.json"
resp=edgar_get(url)
entity_index=resp.json()
filings=pd.DataFrame(entity_index['filings']['recent'])
filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0000320193-21-000114,2021-11-16,2021-11-12,2021-11-16T18:30:25.000Z,34,4,001-36743,211417753,,4609,0,0,xslF345X03/wf-form4_163710541081314.xml,FORM 4
1,0000320193-21-000096,2021-10-19,2021-10-15,2021-10-19T18:30:40.000Z,34,4,001-36743,211332208,,16053,0,0,xslF345X03/wf-form4_163468261390808.xml,FORM 4
2,0000320193-21-000082,2021-09-28,2021-09-26,2021-09-28T18:33:21.000Z,34,4,001-36743,211288784,,5635,0,0,xslF345X03/wf-form4_163286838690995.xml,FORM 4
3,0000320193-21-000049,2021-04-19,2021-04-15,2021-04-19T18:30:35.000Z,34,4,001-36743,21835746,,16126,0,0,xslF345X03/wf-form4_161887141797206.xml,FORM 4
4,0000320193-20-000089,2020-10-19,2020-10-15,2020-10-19T18:30:33.000Z,34,4,001-36743,201247034,,18095,0,0,xslF345X03/wf-form4_160314661733333.xml,FORM 4
5,0000320193-20-000075,2020-09-29,2020-09-27,2020-09-29T18:34:16.000Z,34,4,001-36743,201210091,,5635,0,0,xslF345X03/wf-form4_160141884313005.xml,FORM 4
6,0000320193-20-000056,2020-05-12,2020-05-08,2020-05-12T18:30:32.000Z,34,4,001-36743,20870505,,5074,0,0,xslF345X03/wf-form4_158932261319105.xml,FORM 4
7,0000320193-20-000044,2020-04-17,2020-04-15,2020-04-17T18:30:46.000Z,34,4,001-36743,20800840,,16120,0,0,xslF345X03/wf-form4_158716262832690.xml,FORM 4
8,0000320193-19-000112,2019-10-17,2019-10-15,2019-10-17T18:31:19.000Z,34,4,001-36743,191156367,,17292,0,0,xslF345X03/wf-form4_157135144806488.xml,FORM 4
9,0000320193-19-000100,2019-10-01,2019-09-29,2019-10-01T18:31:55.000Z,34,4,001-36743,191130334,,5671,0,0,xslF345X03/wf-form4_156996908478756.xml,FORM 4
